Let’s switch to the California housing problem and tackle it using a regression neural network. For simplicity, we will use Scikit-Learn’s fetch_california_housing() function to load the data.

since it contains only numerical features (there is no ocean_proximity feature), and there is no missing value. After loading the data, we split it into a training set, a validation set, and a test set, and we scale all the features:

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full)

"""
removing the mean and scaling to unit variance.

The standard score of a sample x is calculated as:

z = (x - u) / s
"""
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

Using the Sequential API to build, train, evaluate, and use a regression MLP to make predictions is quite similar to what we did for classification. 

The main differences are the fact that the output layer has a single neuron (since we only want to predict a single value) and uses no activation function, and the loss function is the mean squared error. 

Since the dataset is quite noisy, we just use a single hidden layer with fewer neurons than before, to avoid overfitting:

In [11]:
import tensorflow as tf
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 1.1404 - val_loss: 0.4894
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4583 - val_loss: 0.4415
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4335 - val_loss: 0.4247
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4089 - val_loss: 0.4133
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4006 - val_loss: 0.4092
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3937 - val_loss: 0.4053
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3898 - val_loss: 0.4018
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3844 - val_loss: 0.3951
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3827 - val_loss: 0.3920
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3773 - val_loss: 0.3916

In [13]:
X_new

array([[-0.19089091,  0.66820013,  0.1595374 , -0.02248317, -0.0741834 ,
        -0.06275659,  1.13195761, -1.12630302],
       [ 2.15559793,  0.50979356,  0.97515386, -0.01055102, -0.81974591,
        -0.05398194, -1.31438175,  1.1497887 ],
       [ 0.1289253 , -0.99506889,  0.29728129, -0.08947918, -0.19928913,
        -0.03361192, -0.05815343, -0.56858518]])

In [12]:
y_pred

array([[1.6585712],
       [4.143652 ],
       [2.5323477]], dtype=float32)

As you can see, the Sequential API is quite easy to use. However, although Sequential models are extremely common, it is sometimes useful to build neural networks with more complex topologies, or with multiple inputs or outputs. For this purpose, Keras offers the Functional API.